In [1]:
import os.path
import pandas as pd

In [2]:
if os.path.isfile('img_ids.txt'):
    with open('img_ids.txt', 'r') as f:
        search_id = f.readline()
        img_ids = [i.rstrip() for i in f.readlines()]
        
df = pd.DataFrame({'id':img_ids, 
                   'year' : [im_id[16:20] for im_id in img_ids], 
                   'month': [im_id[21:23] for im_id in img_ids], 
                   'date' : [im_id[16:26] for im_id in img_ids]})

img_folder = './data'
asset_type = "visual"
item_type = "PSOrthoTile"

def get_dest(r):
    return os.path.join(img_folder, 
                        item_type, 
                        asset_type, 
                        r.year, 
                        r.month, 
                        r.date, 
                        r.id + '.tiff')

df['dest'] = df.apply(get_dest, axis=1)
df['exist'] = df.dest.apply(lambda x: int(os.path.exists(x)))
df.head()

,id,year,month,date,dest,exist
0,2232607_4745521_2019-03-26_106d,2019,03,2019-03-26,./data/PSOrthoTile/visual/2019/03/2019-03-26/2...,1
1,2232607_4745622_2019-03-26_106d,2019,03,2019-03-26,./data/PSOrthoTile/visual/2019/03/2019-03-26/2...,1
2,2232607_4745621_2019-03-26_106d,2019,03,2019-03-26,./data/PSOrthoTile/visual/2019/03/2019-03-26/2...,1
3,2253646_4745522_2019-04-03_1060,2019,04,2019-04-03,./data/PSOrthoTile/visual/2019/04/2019-04-03/2...,0
4,2253646_4745521_2019-04-03_1060,2019,04,2019-04-03,./data/PSOrthoTile/visual/2019/04/2019-04-03/2...,0


In [3]:
## user param
grby = ['year','month'] # 'date' #

In [4]:
total = df.groupby(grby)[["id"]].count().values.transpose().tolist()[0]
loaded = df.groupby(grby)[["exist"]].sum().values.transpose().tolist()[0]
notload = [t-l for t, l in zip(total, loaded)]

In [5]:
from bokeh.core.properties import value
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, HoverTool
output_notebook()

dates = list(df.groupby(grby).groups.keys())
status = ["Loaded", "Not Load"]
colors = ["#718dbf", "#e84d60"]

data = {'date'    : dates,
        'Loaded'  : loaded,
        'Not Load': notload}

p = figure(y_range=FactorRange(*dates), title="Download Status")

p.hbar_stack(status, y='date', height=0.9, color=colors, source=data,
             legend=[value(x) for x in status])

p.x_range.start = 0
p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_right"
p.legend.orientation = "horizontal"

tooltips = HoverTool(
    tooltips=[
        ("Loaded", "@Loaded"),
        ("Not Load", "@{Not Load}"),
        ("Date", "@date")
        ]
    )
p.add_tools(tooltips)


show(p)

Loading BokehJS ...